In [1]:
%pip install ultralytics
%pip install git+https://github.com/openai/CLIP.git

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-i19lzfp0
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-i19lzfp0
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ultralytics import YOLO
import clip
import torch
yolov8 = YOLO("yolov8m-oiv7.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
from PIL import Image
sample_img = Image.open("hand_pen.jpg")
clip_img = preprocess(sample_img).unsqueeze(0).to(device)
question = "What is in the hand?"
candidates = ["a pen", "an apple", "a handgun", "a phone", "a computer", "a cattle", "a cake"]
input_texts = [f"{question} {candidate}" for candidate in candidates]

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(clip_img)
    qa_features = model.encode_text(clip.tokenize(input_texts).to(device))
    similarity = (100.0 * image_features @ qa_features.T).softmax(dim=-1)

In [ ]:
best_index = similarity.squeeze().argmax().item()
best_answer = candidates[best_index]

print(f"{question}\n{best_answer}")

In [ ]:
def VQA(img_path: str, question: str, answers: list):
  sample_img = Image.open(img_path)
  clip_img = preprocess(sample_img).unsqueeze(0).to(device)
  input_texts = [f"{question} {candidate}" for candidate in answers]
  with torch.no_grad():
    image_features = model.encode_image(clip_img)
    qa_features = model.encode_text(clip.tokenize(input_texts).to(device))
    similarity = (100.0 * image_features @ qa_features.T)

  best_index = similarity.squeeze().argmax().item()
  best_answer = answers[best_index]

  print(f"{similarity.squeeze()}\n{question}\n{best_answer}")